## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Corralillo,CU,22.9819,-80.5856,75.90,76,29,8.86,scattered clouds
1,1,Jamestown,US,42.0970,-79.2353,25.00,68,90,11.50,light snow
2,2,Busselton,AU,-33.6500,115.3333,70.00,80,21,4.00,few clouds
3,3,Pitimbu,BR,-7.4706,-34.8086,78.80,89,20,4.61,few clouds
4,4,Atuona,PF,-9.8000,-139.0333,81.12,77,64,13.82,broken clouds
...,...,...,...,...,...,...,...,...,...,...
667,667,Columbia,US,34.0007,-81.0348,46.40,34,1,6.91,clear sky
668,668,Mezen,RU,65.8522,44.2400,27.21,95,100,7.23,overcast clouds
669,669,Ambilobe,MG,-13.2000,49.0500,71.35,92,31,3.18,scattered clouds
670,670,Pemagatsel,BT,27.0319,91.4266,60.75,34,0,3.98,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Corralillo,CU,22.9819,-80.5856,75.90,76,29,8.86,scattered clouds
3,3,Pitimbu,BR,-7.4706,-34.8086,78.80,89,20,4.61,few clouds
4,4,Atuona,PF,-9.8000,-139.0333,81.12,77,64,13.82,broken clouds
5,5,Padang,ID,-0.9492,100.3543,84.20,70,40,9.22,scattered clouds
16,16,Hilo,US,19.7297,-155.0900,78.80,61,75,12.66,heavy intensity rain
...,...,...,...,...,...,...,...,...,...,...
648,648,Itarema,BR,-2.9248,-39.9167,76.46,83,21,9.15,few clouds
653,653,Manta,EC,-0.9500,-80.7333,78.80,69,20,5.01,few clouds
654,654,Nicoya,CR,10.1483,-85.4520,80.60,57,0,24.16,clear sky
658,658,Ternate,ID,0.8000,127.4000,84.27,73,100,5.79,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Corralillo,CU,75.90,scattered clouds,22.9819,-80.5856,
3,Pitimbu,BR,78.80,few clouds,-7.4706,-34.8086,
4,Atuona,PF,81.12,broken clouds,-9.8000,-139.0333,
5,Padang,ID,84.20,scattered clouds,-0.9492,100.3543,
16,Hilo,US,78.80,heavy intensity rain,19.7297,-155.0900,
17,Bambous Virieux,MU,75.20,light rain,-20.3428,57.7575,
19,Mount Gambier,AU,87.80,clear sky,-37.8333,140.7667,
21,Manzanillo,CU,76.96,scattered clouds,20.3433,-77.1167,
22,Zabid,YE,77.18,clear sky,14.1951,43.3152,
25,Kapaa,US,78.80,broken clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Corralillo,CU,75.90,scattered clouds,22.9819,-80.5856,Hostal De Pedri
3,Pitimbu,BR,78.80,few clouds,-7.4706,-34.8086,Reserva do Abiaí
4,Atuona,PF,81.12,broken clouds,-9.8000,-139.0333,Villa Enata
5,Padang,ID,84.20,scattered clouds,-0.9492,100.3543,Mercure Padang
16,Hilo,US,78.80,heavy intensity rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
648,Itarema,BR,76.46,few clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
653,Manta,EC,78.80,few clouds,-0.9500,-80.7333,Balandra Hotel
654,Nicoya,CR,80.60,clear sky,10.1483,-85.4520,Guayacan Real
658,Ternate,ID,84.27,overcast clouds,0.8000,127.4000,Grand Dafam Bela Ternate


In [10]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))